In [17]:
import numpy as np
from sensor_msgs.msg import PointCloud2, PointField
import rospy
import math
import sys
import std_msgs.msg
import sensor_msgs.point_cloud2 as pc2
from IPython.display import clear_output
import time
import threading

take_arg_max = False
wait = False

load_file = "/home/juli/Downloads/multi_pcl.npy"
#load_file = "/home/juli/Downloads/multi_pcl_sync_sampled_labelled_projected_refined.npy"

frames = np.load(load_file, encoding="bytes")

In [18]:
def signal_user_input():
    global wait
    while (True):
        i = input("hit enter to stop things")   # I have python 2.7, not 3.x
        wait = not wait
        # thread exits here

print(len(frames))
print(frames[0].shape)
print(len(frames[0][0]))
print(frames)

303
(320751, 4)
4
[array([[-3.7195492e+00, -5.8637595e+00,  9.8863935e+00,  1.3827530e-38],
       [-3.6738136e+00, -5.8259292e+00,  9.8226109e+00,  1.0969823e-38],
       [-3.6998146e+00, -5.9020848e+00,  9.9510107e+00,  7.1908766e-39],
       ...,
       [ 8.1830066e-01,  9.3176258e-01,  1.5535761e+00,  0.0000000e+00],
       [ 8.2257831e-01,  9.3271434e-01,  1.5551630e+00,  0.0000000e+00],
       [ 8.2517719e-01,  9.3176258e-01,  1.5535761e+00,  0.0000000e+00]],
      dtype=float32)
 array([[-3.7195492e+00, -5.8637595e+00,  9.8863935e+00,  1.3827530e-38],
       [-3.6738136e+00, -5.8259292e+00,  9.8226109e+00,  1.0969823e-38],
       [-3.6998146e+00, -5.9020848e+00,  9.9510107e+00,  7.1908766e-39],
       ...,
       [ 8.1830066e-01,  9.3176258e-01,  1.5535761e+00,  0.0000000e+00],
       [ 8.2257831e-01,  9.3271434e-01,  1.5551630e+00,  0.0000000e+00],
       [ 8.2517719e-01,  9.3176258e-01,  1.5535761e+00,  0.0000000e+00]],
      dtype=float32)
 array([[-3.7684906e+00, -5.9409146e

In [13]:
# Publish Labeled Pointcloud
if __name__ == '__main__':
    # Start thread
    threading.Thread(target = signal_user_input).start()
    # Read in file
    cloud_msg = PointCloud2() 
    cloud_msg.fields = [
        PointField('x', 0, PointField.FLOAT32, 1),
        PointField('y', 4, PointField.FLOAT32, 1),
        PointField('z', 8, PointField.FLOAT32, 1),
        PointField('intensity', 12, PointField.FLOAT32, 1),
        PointField('r', 16, PointField.FLOAT32, 1),
        PointField('l_none', 20, PointField.FLOAT32, 1),
        PointField('l_dust', 24, PointField.FLOAT32, 1),
        PointField('l_smoke', 28, PointField.FLOAT32, 1)
    ]
    
    # Create publisher
    pcl_pub = rospy.Publisher("/velodyne_points_dual", PointCloud2, queue_size=10)
    rospy.init_node('pc2_publisher', anonymous=True)
    rate = rospy.Rate(10) # 10hz
    
    rospy.loginfo("Initializing pc2 publisher node...")
    # Give time to roscore to make the connections
    rospy.sleep(1.)
    
    # Header
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = 'map'
    
    # Publish    
    # rospy.loginfo("happily publishing sample pointcloud.. !")
    time.sleep(5)
    for i, frame in enumerate(frames):
        while wait:
            time.sleep(0.01)
        # Add labels
        #frame_and_labels = np.zeros((frame.shape[0],frame.shape[1]+1))
        #frame_and_labels[:,:-1] = frame
        #rospy.loginfo("Frame: %s", i)
        #clear_output(wait=True)
        print(i), # Because of input
        points = np.asarray(frame[:, :5])
        labels = np.asarray(frame[:, -3:])
        if take_arg_max:
            max_indices_true = np.argmax(labels, axis = 1)
            print(np.sum(max_indices_true >= 1))
            labels[:,:] = 0
            for j in range(
                len(labels)):
                labels[j,max_indices_true[j]] = 1
        frame_and_labels = np.concatenate((points, labels), axis = 1)
        #frame_and_labels[:,5] = np.round(frame_and_labels[:,5])
        # Create pcl from points
        scaled_polygon_pcl = pc2.create_cloud(header, cloud_msg.fields, frame_and_labels)
        pcl_pub.publish(scaled_polygon_pcl)
        rate.sleep()

[INFO] [1554922091.731559]: Initializing pc2 publisher node...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [9]:
# Publish Unlabeled Pointcloud
if __name__ == '__main__':
    # Read in file
    cloud_msg = PointCloud2() 
    cloud_msg.fields = [
        PointField('x', 0, PointField.FLOAT32, 1),
        PointField('y', 4, PointField.FLOAT32, 1),
        PointField('z', 8, PointField.FLOAT32, 1),
        PointField('intensity', 12, PointField.FLOAT32, 1),
        PointField('r', 16, PointField.FLOAT32, 1)
    ]
    
    # Create publisher
    pcl_pub = rospy.Publisher("/velodyne_points", PointCloud2, queue_size=10)
    rospy.init_node('pc2_publisher', anonymous=True)
    rate = rospy.Rate(10) # 10hz
    
    rospy.loginfo("Initializing pc2 publisher node...")
    # Give time to roscore to make the connections
    rospy.sleep(1.)
    
    # Header
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = 'map'
    
    # Publish    
    rospy.loginfo("happily publishing sample pointcloud.. !")
    for i, frame in enumerate(frames):
        # Add labels
        #frame_and_labels = np.zeros((frame.shape[0],frame.shape[1]+1))
        #frame_and_labels[:,:-1] = frame
        rospy.loginfo("Frame: %s", i)
        clear_output(wait=True)
        points = np.asarray(frame[:, :5])
        #frame_and_labels[:,5] = np.round(frame_and_labels[:,5])
        # Create pcl from points
        scaled_polygon_pcl = pc2.create_cloud(header, cloud_msg.fields, points)
        pcl_pub.publish(scaled_polygon_pcl)
        rate.sleep()

[INFO] [1581187012.927770]: Frame: 302


In [69]:
print(np.argmax([[1,2],[3,1]],axis = 1))

[1 0]
